In [4]:
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Load your local model (e.g., llama3)
llm = Ollama(model="llama3.2:1b")  # or "mistral" or another Ollama model

# Create a simple prompt template
template = "What is the capital of {country}?"
prompt = PromptTemplate(input_variables=["country"], template=template)

# Create a chain that combines prompt + LLM
chain = LLMChain(llm=llm, prompt=prompt)

# Ask a question
response = chain.run("France")
print(response)

The capital of France is Paris.


In [17]:
import psycopg2
from psycopg2.extras import RealDictCursor
from langchain.docstore.document import Document
from langchain_community.llms import Ollama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA

In [6]:
def get_db_connection():
    return psycopg2.connect(
        dbname='Econ',
        user='postgres',
        password='123456789',
        host='localhost',
        cursor_factory=RealDictCursor
    )

In [23]:
def get_data():
    conn = get_db_connection()
    cursor = conn.cursor(cursor_factory=RealDictCursor)
    cursor.execute('SELECT name,price FROM public.produce ORDER BY id ASC')
    processing = cursor.fetchall()
    cursor.close()
    conn.close()
    return processing

In [24]:
data = get_data()
docs = []
for idx, row in enumerate(data):
    content = "\n".join([f"{key}: {value}" for key, value in row.items()])
    print(content)
    docs.append(Document(page_content=content, metadata={"row": idx}))


name: vivo V50 Lite
price: 9,999
name: iPhone 16 Pro Max
price: 48,900
name: Samsung Galaxy S25 Ultra
price: 46,900


In [25]:
# Step 3: Split text if necessary
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(docs)

In [26]:
split_docs

[Document(metadata={'row': 0}, page_content='name: vivo V50 Lite\nprice: 9,999'),
 Document(metadata={'row': 1}, page_content='name: iPhone 16 Pro Max\nprice: 48,900'),
 Document(metadata={'row': 2}, page_content='name: Samsung Galaxy S25 Ultra\nprice: 46,900')]

In [19]:
# Step 4: Embed documents
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(split_docs, embeddings)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_15944\986379964.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [28]:
# Step 5: Load your local Ollama model
llm = Ollama(model="gemma3:4b-it-qat")

# Step 6: Set up the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

In [29]:
# Step 7: Ask questions
print("📊 CSV QA Bot is ready! Ask a question (type 'exit' to stop):\n")
while True:
    query = input("You: ")
    if query.lower() in ["exit", "quit"]:
        print("👋 Goodbye!")
        break

    result = qa_chain(query)
    print("\n🧠 Answer:", result["result"])
    print("📄 Rows matched:", [doc.metadata.get("row", "unknown") for doc in result["source_documents"]])
    print("-" * 50)

📊 CSV QA Bot is ready! Ask a question (type 'exit' to stop):


🧠 Answer: Samsung Galaxy S25 Ultra, iPhone 16 Pro Max

📄 Rows matched: [2, 0, 1]
--------------------------------------------------

🧠 Answer: The vivo V50 Lite is the cheapest at 9,999.
📄 Rows matched: [2, 0, 1]
--------------------------------------------------

🧠 Answer: The iPhone 16 Pro Max is the most expensive at 48,900.
📄 Rows matched: [2, 0, 1]
--------------------------------------------------
👋 Goodbye!
